https://colab.research.google.com/github/sanchit-gandhi/notebooks/blob/main/fine_tune_whisper.ipynb

https://colab.research.google.com/github/Vaibhavs10/fast-whisper-finetuning/blob/main/Whisper_w_PEFT.ipynb

https://huggingface.co/docs/transformers/model_doc/whisper

In [1]:
from datasets import  Dataset, DatasetDict, Audio, load_dataset
import pandas as pd
import os
from IPython.display import Audio as IPAudio
from tqdm import tqdm
import librosa
import numpy as np
from jiwer import wer

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
LANG = 'hi'
# DATASET_ROOT_PATH = '/mnt/coding-nvme/datasets/audio_ds/cv-corpus-14.0-2023-06-23/hi'
DATASER_NAME = 'mozilla-foundation/common_voice_13_0'
MODEL_DIR_FP32 = 'models/raw_hf/whisper-medium'
OUTPUT_DIR = 'models/output/whisper_medium_fp32_hindi_transformers'

In [4]:
# train = pd.read_csv(os.path.join(DATASET_ROOT_PATH, 'train.tsv'), sep='\t')
# valid = pd.read_csv(os.path.join(DATASET_ROOT_PATH, 'dev.tsv'), sep='\t')
# test = pd.read_csv(os.path.join(DATASET_ROOT_PATH, 'test.tsv'), sep='\t')

In [5]:
# train = train.merge(valid, how='outer')
# train = train[['path', 'sentence']]
# train = train.rename(columns={'sentence': 'text', 'path': 'audio'})

# test = test[['path', 'sentence']]
# test = test.rename(columns={'sentence': 'text', 'path': 'audio'})

In [6]:
# print(train.shape, test.shape)

In [7]:
# data = DatasetDict({'train': Dataset.from_pandas(train), 'test': Dataset.from_pandas(test)})

In [8]:
data = DatasetDict()
data['train'] = load_dataset(DATASER_NAME, LANG, split="train+validation")
data['test'] = load_dataset(DATASER_NAME, LANG, split="test")

Found cached dataset common_voice_13_0 (/home/jarvis/.cache/huggingface/datasets/mozilla-foundation___common_voice_13_0/hi/13.0.0/2506e9a8950f5807ceae08c2920e814222909fd7f477b74f5d225802e9f04055)
Found cached dataset common_voice_13_0 (/home/jarvis/.cache/huggingface/datasets/mozilla-foundation___common_voice_13_0/hi/13.0.0/2506e9a8950f5807ceae08c2920e814222909fd7f477b74f5d225802e9f04055)


In [9]:
data = data.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"]
)

In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 6760
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2947
    })
})

In [11]:
# # Only keeping 1k files in test set

# # Get the number of samples in the test set
# num_samples_test = len(data['test'])
# # Determine the number of samples to move from test to train
# num_samples_to_move = num_samples_test - 1000
# # Move the samples from test to train
# data['train'] = data['train'](data['test'].shuffle().select(range(num_samples_to_move)))
# # Update the test set to keep only 1000 samples
# data['test'] = data['test'].select(range(num_samples_to_move, num_samples_test))
# # Verify the number of samples in train and test after the split
# num_samples_train = len(data['train'])
# num_samples_test = len(data['test'])
# print(f"Train samples: {num_samples_train}")
# print(f"Test samples: {num_samples_test}")

In [12]:
from transformers import (
    WhisperForConditionalGeneration, WhisperProcessor, WhisperConfig, WhisperTokenizer, WhisperFeatureExtractor
)
import torch

2023-07-04 02:44:49.816031: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
processor = WhisperProcessor.from_pretrained(MODEL_DIR_FP32, language=LANG, task='transcribe')
tokenizer = WhisperTokenizer.from_pretrained(MODEL_DIR_FP32, language=LANG, task='transcribe')
feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_DIR_FP32)
config = WhisperConfig.from_pretrained(MODEL_DIR_FP32)

In [19]:
model = WhisperForConditionalGeneration(
        config=config
    ).from_pretrained(
                    pretrained_model_name_or_path = MODEL_DIR_FP32, 
                    # torch_dtype = config.torch_dtype,
                    load_in_8bit = True,
                    low_cpu_mem_usage = True,
                )

In [20]:
# Move model to GPU
if model.device.type != 'cuda':
    print('Moving model to GPU')
    model = model.to('cuda')
    model.eval()

else:
    print('Model is already on GPU')
    model.eval()

Model is already on GPU


In [21]:
print('dtype of model acc to config: ', config.torch_dtype)
print('dtype of loaded model: ', model.dtype)

dtype of model acc to config:  torch.float32
dtype of loaded model:  torch.float16


In [ ]:
# def preprocess_audio(row):
#     audio, sr = librosa.load(os.path.join(DATASET_ROOT_PATH, 'clips', row['audio']))
#     # remove audios with duration less than 1 second
#     if len(audio) < sr * 1:
#         return {
#             'audio': None,
#             'text': None,
#         }
#     # remove audios with duration more than 25 seconds
#     elif len(audio) > sr * 25:
#         return {
#             'audio': None,
#             'text': None,
#         }
#     else:
#         return {
#             'audio': feature_extractor(
#                 librosa.resample(y=audio, orig_sr=sr, target_sr=16000),
#                 sampling_rate=16000
#                 ).input_features[0],
#             'text': tokenizer(
#                 row['text']
#                 ).input_ids,
#         }

In [ ]:
# train_ds = train_ds.map(preprocess_audio, num_proc=1)
# test_ds = test_ds.map(preprocess_audio, num_proc=1)

In [22]:
data['train'][0]

{'audio': {'path': '/home/jarvis/.cache/huggingface/datasets/downloads/extracted/6fc68b039af578e2afaa36faf74b7e012f1b87666c6efdb117a8a9c732edbc21/hi_train_0/common_voice_hi_26008353.mp3',
  'array': array([ 5.81611368e-26, -1.48634016e-25, -9.37040538e-26, ...,
          1.06425901e-07,  4.46416450e-08,  2.61450239e-09]),
  'sampling_rate': 48000},
 'sentence': 'हमने उसका जन्मदिन मनाया।'}

In [23]:
data = data.cast_column('audio', Audio(sampling_rate=16000))

In [24]:
IPAudio(data['train'][0]['audio']['array'], rate=16000)

In [25]:
def prepare_ds(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [26]:
data = data.map(prepare_ds, remove_columns=data.column_names["train"], num_proc=1)

Loading cached processed dataset at /home/jarvis/.cache/huggingface/datasets/mozilla-foundation___common_voice_13_0/hi/13.0.0/2506e9a8950f5807ceae08c2920e814222909fd7f477b74f5d225802e9f04055/cache-9515d7081b9f1211.arrow


Map:   0%|          | 0/2947 [00:00<?, ? examples/s]

In [27]:
data['train']

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 6760
})

In [28]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [29]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

## Using FP32

In [20]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer_v = 100 * wer(hypothesis=pred_str, references=label_str)

    return {"wer": wer_v}

In [21]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [31]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4,  #16
    gradient_accumulation_steps=4,  #1   # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100,   #500
    max_steps=10000,    #4000
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    # ... more args
    seed=42,
    lr_scheduler_type="linear",
    optim="adamw_hf",   #adam
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-8,
    adafactor=False,
    auto_find_batch_size=False,
    bf16=False,
    bf16_full_eval=False,
    data_seed=None,
    dataloader_drop_last=False,
    dataloader_num_workers=0,
    dataloader_pin_memory=True,
    ddp_backend=None,
)

In [32]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [29]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

## using PEFT and LoRA

In [39]:
from peft import prepare_model_for_int8_training

model = prepare_model_for_int8_training(model, output_embedding_layer_name="proj_out")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 from peft import prepare_model_for_int8_training                                             │
│   2                                                                                              │
│ ❱ 3 model = prepare_model_for_int8_training(model, output_embedding_layer_name="proj_out")       │
│   4                                                                                              │
│                                                                                                  │
│ /home/jarvis/.local/lib/python3.10/site-packages/peft/utils/other.py:80 in                       │
│ prepare_model_for_int8_training                                                                  │
│                                                                                                  │
│    77 │   │   "prepare_model_for_int8_training is deprecated and will be removed in a future v   │
│    78 │   │   FutureWarning,                                                                     │
│    79 │   )                                                                                      │
│ ❱  80 │   return prepare_model_for_kbit_training(*args, **kwargs)                                │
│    81                                                                                            │
│    82                                                                                            │
│    83 # copied from transformers.models.bart.modeling_bart                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: prepare_model_for_kbit_training() got an unexpected keyword argument 'output_embedding_layer_name'

In [31]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

In [32]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 9437184 || all params: 773295104 || trainable%: 1.2203858463844612


In [33]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # increase by 2x for every 2x decrease in batch size
    learning_rate=6.25*1e-6, #1e-5,
    warmup_steps=100,
    num_train_epochs=1,
    max_steps=100, # only for testing purposes, remove this from your final run :)
    evaluation_strategy="steps",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=100,
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
    optim="adamw_hf",   #adam
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-8,
)

In [35]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

# This callback helps to save only the adapter weights and remove the base model weights.
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [36]:
trainer.train()

/home/jarvis/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/jarvis/.netrc


  0%|          | 0/100 [00:00<?, ?it/s]

/home/jarvis/.local/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /home/jarvis/.local/lib/python3.10/site-packages/transformers/trainer.py:1645 in train           │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /home/jarvis/.local/lib/python3.10/site-packages/transformers/trainer.py:1938 in                 │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /home/jarvis/.local/lib/python3.10/site-packages/transformers/trainer.py:2759 in training_step   │
│                                                                                                  │
│   2756 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2757 │   │                                                                                     │
│   2758 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2759 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2760 │   │                                                                                     │
│   2761 │   │   if self.args.n_gpu > 1:                                                           │
│   2762 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /home/jarvis/.local/lib/python3.10/site-packages/transformers/trainer.py:2784 in compute_loss    │
│                                                                                                  │
│   2781 │   │   │   labels = inputs.pop("labels")           